In [608]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor


In [679]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/PratikBorkar04/consignment_pricing/main/notebook/dataset/xgb_dataset.csv")


In [680]:
df.head(1)

,country,fulfill_via,vendor,shipment_mode,unit_of_measure,line_item_quantity,line_item_value,pack_price,unit_price,weight,freight_cost,location,distance
0,Vietnam,Direct Drop,EXW,Air,240,1000,6200,6,0.03,358,3521,India,3157


In [681]:
df.isnull().sum()

country               0
fulfill_via           0
vendor                0
shipment_mode         0
unit_of_measure       0
line_item_quantity    0
line_item_value       0
pack_price            0
unit_price            0
weight                0
freight_cost          0
location              0
distance              0
dtype: int64

In [682]:
df.shape

(6253, 13)

In [683]:
'''# Drop fullfill column to increase the accuracy
df.drop('fulfill_via', axis=1, inplace=True)
df.to_csv("CatBoosting_dataset.csv",index = False)
'''

'# Drop fullfill column to increase the accuracy\ndf.drop(\'fulfill_via\', axis=1, inplace=True)\ndf.to_csv("CatBoosting_dataset.csv",index = False)\n'

In [685]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
categorical_columns = ['country', 'vendor', 'shipment_mode', 'location']
for col in categorical_columns:
    unique_values = df[col].unique().reshape(-1, 1)  # Reshape to 2D array
    encoded_values = ordinal_encoder.fit_transform(unique_values)
    
    mapping = dict(zip(unique_values.flatten(), encoded_values.flatten()))
    df[col] = df[col].map(mapping)
    
    print(f"Original unique values in '{col}':", unique_values.flatten())
    print(f"Encoded unique values in '{col}':", encoded_values.flatten())
    print("--" * 18)

Original unique values in 'country': [ 9.  4.  7. 10.  5. 11.  1.  2.  6.  8.  3.  0.]
Encoded unique values in 'country': [ 9.  4.  7. 10.  5. 11.  1.  2.  6.  8.  3.  0.]
------------------------------------
Original unique values in 'vendor': ['EXW' 'CIP' 'FCA' 'DDP' 'RDC' 'DAP']
Encoded unique values in 'vendor': [3. 0. 4. 2. 5. 1.]
------------------------------------
Original unique values in 'shipment_mode': ['Air' 'Truck' 'Air Charter' 'Ocean']
Encoded unique values in 'shipment_mode': [0. 3. 1. 2.]
------------------------------------
Original unique values in 'location': ['India' 'Germany' 'France' 'Thailand' ' Nashik' 'Switzerland' ' Haarlem'
 ' Port Elizabeth' 'Canada' 'South Africa' 'Puerto Rico' ' Latina'
 'Poland' 'Cyprus' 'China']
Encoded unique values in 'location': [ 9.  8.  7. 14.  2. 13.  0.  3.  4. 12. 11.  1. 10.  6.  5.]
------------------------------------


In [648]:
from sklearn import preprocessing
  
# label_encoder object knows 

label_encoder = preprocessing.LabelEncoder()
categorical_columns = ['country', 'vendor', 'shipment_mode', 'location']
for i in categorical_columns:
    print(df[i].unique())
    df[i]= label_encoder.fit_transform(df[i])
    # Encode labels in column 'species'.
    print(df[i].unique())
    print("--"*18)


['Vietnam' 'Nigeria' 'Tanzania' 'Zambia' 'Rwanda' 'Zimbabwe'
 "Côte d'Ivoire" 'Haiti' 'South Africa' 'Uganda' 'Mozambique' 'Congo, DRC']
[ 9  4  7 10  5 11  1  2  6  8  3  0]
------------------------------------
['EXW' 'CIP' 'FCA' 'DDP' 'RDC' 'DAP']
[3 0 4 2 5 1]
------------------------------------
['Air' 'Truck' 'Air Charter' 'Ocean']
[0 3 1 2]
------------------------------------
['India' 'Germany' 'France' 'Thailand' ' Nashik' 'Switzerland' ' Haarlem'
 ' Port Elizabeth' 'Canada' 'South Africa' 'Puerto Rico' ' Latina'
 'Poland' 'Cyprus' 'China']
[ 9  8  7 14  2 13  0  3  4 12 11  1 10  6  5]
------------------------------------


In [649]:
df.head()

,country,vendor,shipment_mode,unit_of_measure,line_item_quantity,line_item_value,pack_price,unit_price,weight,freight_cost,location,distance
0,9,3,0,240,1000,6200,6,0.03,358,3521,9,3157
1,9,3,0,60,31920,127360,3,0.07,1855,15007,9,3157
2,9,3,0,60,38000,121600,3,0.05,7590,44450,9,3157
3,4,3,0,240,416,2225,5,0.02,504,4920,9,7633
4,7,3,0,60,16667,60834,3,0.06,1478,5212,9,5659


In [616]:
#Preapare X and y variables
X = df.drop(columns=['freight_cost'],axis=1)    

In [617]:
X.head()

,country,vendor,shipment_mode,unit_of_measure,line_item_quantity,line_item_value,pack_price,unit_price,weight,location,distance
0,9,3,0,240,1000,6200,6,0.03,358,9,3157
1,9,3,0,60,31920,127360,3,0.07,1855,9,3157
2,9,3,0,60,38000,121600,3,0.05,7590,9,3157
3,4,3,0,240,416,2225,5,0.02,504,9,7633
4,7,3,0,60,16667,60834,3,0.06,1478,9,5659


In [618]:
y = df['freight_cost']


In [619]:
y

0        3521
1       15007
2       44450
3        4920
4        5212
        ...  
6248     9181
6249     9181
6250     9181
6251     9181
6252     9181
Name: freight_cost, Length: 6253, dtype: int64

In [620]:
'''# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)'''
'''X = preprocessor.fit_transform(X)
X.shape'''

'X = preprocessor.fit_transform(X)\nX.shape'

In [621]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape


((5002, 11), (1251, 11))

In [622]:
print(X_train)
print(X_test)

      country  vendor  shipment_mode  unit_of_measure  line_item_quantity   
478         6       2              0               60                1000  \
5099        4       5              1              120               43169   
1203        4       5              0               30               13473   
5645        1       5              0              240                3500   
142         4       3              0               60                4632   
...       ...     ...            ...              ...                 ...   
3772        1       5              3               60               40035   
5191        4       5              1               60                 912   
5226        1       5              0               30                6464   
5390        4       5              0               90                6654   
860         4       5              0               60                 451   

      line_item_value  pack_price  unit_price  weight  location  distance  

In [623]:
#Create an Evaluate Function to give all metrics after model Training
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [624]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')


Linear Regression
Model performance for Training set
- Root Mean Squared Error: 9098.3978
- Mean Absolute Error: 4452.0787
- R2 Score: 0.2955
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 10967.9721
- Mean Absolute Error: 4658.0867
- R2 Score: 0.1082


Lasso
Model performance for Training set
- Root Mean Squared Error: 9098.4010
- Mean Absolute Error: 4451.8990
- R2 Score: 0.2955
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 10967.7062
- Mean Absolute Error: 4658.0058
- R2 Score: 0.1083


Ridge
Model performance for Training set
- Root Mean Squared Error: 9098.3978
- Mean Absolute Error: 4452.0492
- R2 Score: 0.2955
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 10967.9620
- Mean Absolute Error: 4658.0614
- R2 Score: 0.1082


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 8113.6404
- Mean Absolute Error: 3589.38

In [625]:
#Results
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
7,CatBoosting Regressor,0.608919
5,Random Forest Regressor,0.484859
6,XGBRegressor,0.455895
4,Decision Tree,0.297873
8,AdaBoost Regressor,0.168108
3,K-Neighbors Regressor,0.155706
1,Lasso,0.108251
2,Ridge,0.108209
0,Linear Regression,0.108207


In [626]:
#catBoostRegressor
CatBoosting_Regressor = CatBoostRegressor(verbose=False)
CatBoosting_Regressor = CatBoosting_Regressor.fit(X_train, y_train)
y_pred = CatBoosting_Regressor.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 60.89


In [627]:
pred_df=pd.DataFrame({'Actual Value':y_test,'Predicted Value':y_pred,'Difference':y_test-y_pred})
pred_df


,Actual Value,Predicted Value,Difference
1703,6292,12181.001846,-5889.001846
5449,9181,8077.588173,1103.411827
5058,5933,4097.977158,1835.022842
1149,651,2736.243698,-2085.243698
432,9181,9047.200177,133.799823
...,...,...,...
416,9181,9358.023535,-177.023535
6111,20520,12897.164194,7622.835806
3185,721,-161.521385,882.521385
2025,4726,5793.829780,-1067.829780
